[](http://)

In [ ]:
!pip install -q /kaggle/input/loguru-lib-ds/loguru-0.5.3-py3-none-any.whl
!pip install -q /kaggle/input/bbox-lib-ds

In [ ]:
import numpy as np
from tqdm.notebook import tqdm

tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys

sys.path.append("../input/tensorflow-great-barrier-reef")
import torch
from PIL import Image
import greatbarrierreef
from bbox.utils import *


In [ ]:
# Train Data
df = pd.read_csv(f"/kaggle/input/tensorflow-great-barrier-reef/train.csv")
df["image_path"] = (
    f"/kaggle/input/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype(str)
    + "/"
    + df.video_frame.astype(str)
    + ".jpg"
)
df["annotations"] = df["annotations"].progress_apply(eval)
display(df.head(2))


In [ ]:
df["num_bbox"] = df["annotations"].progress_apply(lambda x: len(x))
data = (df.num_bbox > 0).value_counts() / len(df) * 100
print(f"No BBox: {data[0]:0.2f}% | With BBox: {data[1]:0.2f}%")


In [ ]:
def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes


def get_imgsize(row):
    row["width"], row["height"] = imagesize.get(row["image_path"])
    return row


np.random.seed(32)
colors = [
    (np.random.randint(255), np.random.randint(255), np.random.randint(255))
    for idx in range(1)
]

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
def load_model(ckpt_path, conf=0.25, iou=0.50):
    model = torch.hub.load(
        "/kaggle/input/yolov5-lib-ds",
        "custom",
        path=ckpt_path,
        source="local",
        force_reload=True,
    )  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou = iou  # NMS IoU threshold
    model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
    return model

In [ ]:
def predict(model, img, size=768, augment=False):
    height, width = img.shape[:2]
    results = model(img, size=size, augment=augment)  # custom inference size
    preds = results.pandas().xyxy[0]
    bboxes = preds[["xmin", "ymin", "xmax", "ymax"]].values
    if len(bboxes):
        bboxes = voc2coco(bboxes, height, width).astype(int)
        confs = preds.confidence.values
        return bboxes, confs
    else:
        return [], []


def format_prediction(bboxes, confs):
    annot = ""
    if len(bboxes) > 0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf = confs[idx]
            annot += f"{conf} {xmin} {ymin} {w} {h}"
            annot += " "
        annot = annot.strip(" ")
    return annot


def show_img(img, bboxes, bbox_format="yolo"):
    names = ["starfish"] * len(bboxes)
    labels = [0] * len(bboxes)
    img = draw_bboxes(
        img=img,
        bboxes=bboxes,
        classes=names,
        class_ids=labels,
        class_name=True,
        colors=colors,
        bbox_format=bbox_format,
        line_thickness=2,
    )
    return Image.fromarray(img).resize((800, 400))

In [ ]:
CKPT_PATH = "../input/yolov5s6/f2_sub2.pt"
CONF = 0.30
IOU = 0.50
IMG_SIZE = 6400
AUGMENT = True

In [ ]:
env = greatbarrierreef.make_env()
iter_test = env.iter_test()

In [ ]:
model = load_model(CKPT_PATH, conf=CONF, iou=IOU)
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    bboxes, confs = predict(model, img, size=IMG_SIZE, augment=AUGMENT)
    annot = format_prediction(bboxes, confs)
    pred_df["annotations"] = annot
    env.predict(pred_df)
    if idx < 3:
        display(show_img(img, bboxes, bbox_format="coco"))

In [ ]:
sub_df = pd.read_csv("submission.csv")
sub_df.head()